In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import traceback
import time
import os 
import os.path as osp
import glob

In [2]:
os.getcwd()

'D:\\Project- Electro\\Predict_Social_Behavior\\src\\correct_simaba_files'

In [3]:
from pykalman import KalmanFilter
from scipy.signal import savgol_filter

In [100]:
# root_folder = osp.join("..","..","..","Summary of free interaction - 1-2023")
# video_folder = osp.join(root_folder,'cropped_raw_video')

# video_folder = osp.join('cropped_videos','videos')
# all_video_files = glob.glob(osp.join(video_folder,'*.avi'))

# all_video_files = glob.glob(osp.join(video_folder,'*.avi'))

# pose_data_folder = osp.join('project','project_folder','csv','features_extracted')

# save_folder = 'annotated_temp'

# # filename = 'cropped_Rat14-probe14-day1-free_2018-09-10-112737-0000.avi'
# filename = osp.split(all_video_files[2])[-1]


# csv_filename = filename.replace('.avi','.csv')
# df_path = osp.join(pose_data_folder,filename)
# df = pd.read_csv(df_path)

# video_path = osp.join(video_folder,filename).replace('.csv','.avi')
# save_path =  osp.join(save_folder,filename).replace('.csv','_annotated.csv')


In [4]:
root_folder = osp.join("..","..","..","Summary of free interaction - 1-2023")
video_folder = osp.join(root_folder,'cropped_raw_video')

all_video_files = glob.glob(osp.join(video_folder,'*.avi'))

file_ind = 0
video_path = all_video_files[file_ind]
video_filename = osp.split(all_video_files[file_ind])[-1]


# pose_data_folder = osp.join('project','project_folder','csv','features_extracted')
pose_data_folder = osp.join(root_folder, 'video_pose_estimation')

# csv_filename = video_filename.replace('.avi','.csv')
csv_filename = video_filename.replace('.avi','_reconstructed.csv')

csv_path = osp.join(pose_data_folder, csv_filename)
df = pd.read_csv(csv_path)
# current_dir = os.getcwd()
# os.chdir(osp.join(".."))
# from utils.helper_functions import fix_df_col_names
# os.chdir(current_dir)
# df = fix_df_col_names(df)

In [5]:
pose_data_folder_original = osp.join('project','project_folder','csv','features_extracted')
csv_filename_original = video_filename.replace('.avi','.csv')

csv_path_original = osp.join(pose_data_folder_original, csv_filename_original)

df_original =  pd.read_csv(csv_path_original)
current_dir = os.getcwd()
os.chdir(osp.join(".."))
from utils.helper_functions import fix_column_names
os.chdir(current_dir)
df_original = fix_column_names(df_original)

In [6]:
video_filename

'cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000.avi'

In [7]:

# pose_data_folder = osp.join('project','project_folder','csv','features_extracted')

# save_folder = 'annotated_temp'

# # filename = 'cropped_Rat14-probe14-day1-free_2018-09-10-112737-0000.avi'
# filename = osp.split(all_video_files[2])[-1]


# filename = filename.replace('.avi','.csv')
# df_path = osp.join(pose_data_folder,filename)
# df = pd.read_csv(df_path)

# video_path = osp.join(video_folder,filename).replace('.csv','.avi')
# save_path =  osp.join(save_folder,filename).replace('.csv','_annotated.csv')


In [8]:
action_map = {'1': 'subject_investigate_stim',
              '2': 'stimulus_investigate_subject',
              '3': 'both_investigate_each_other',
              '4': 'subject_head_scratch',

              'q': 'subject_approach_stim',
              'w': 'stim_approach_subject',
              'e': 'both_approach',
              'r': 'subject_follow_stim',
              't': 'stim_follow_subject',
              
              'y': 'both_idle_close',

              'a': 'subject_walk',
              's': 'stim_walk',
              'd': 'both_walk',
              '-': 'unlabeled'
              }

In [9]:
# # df_path = 'cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000.csv'
# df_path = 'cropped_Rat10-probe13-day1-free_2018-07-15-103307-0000.csv'
# folder_path = 'samples'
# df_full_path = osp.join(folder_path,df_path)
# df = pd.read_csv(df_full_path)


In [10]:
all_scores = [a for a in df.columns if any(substring in a for substring in ['_p'])]
# all_scores

In [11]:
all_joints = [a for a in df.columns if any(substring in a for substring in ['_x', '_y'])]
score_joints = [a.replace('_x','_p') for a in all_joints if '_x' in a]
df_pose = df[all_joints]
df_score = df[score_joints]


In [12]:
import cv2
import imageio
# from tmdq import tmdp

def load_video_frames_2(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames



def load_video_frames(video_path):
    vid = imageio.get_reader(video_path, 'ffmpeg')
    frames = [frame for frame in vid]

    return frames

def get_array_from_csv_file(csv_path):
    # Check if the CSV file exists
    if os.path.exists(csv_path):
        # If it exists, read the CSV as a dataframe
        df = pd.read_csv(csv_path)
        array = df['action'].values
        array = [str(a) for a in array]
        return array
    else:
        return None
    
    

def generate_distinct_colors(num_groups, num_classes):
    # Calculate the base hue range for each group
    base_hues = np.linspace(0, 1, num_groups + 1)[:-1]

    # Calculate the class hue offset within each group
    class_hue_offsets = np.linspace(0, 1, num_classes + 1)[:-1] / (num_classes * 0.25)

    # Generate distinct RGB colors for each group and class
    colors = []
    for base_hue in base_hues:
        group_colors = []
        for hue_offset in class_hue_offsets:
            hue = (base_hue + hue_offset) % 1
            rgb_color = plt.cm.hsv(hue)
            group_colors.append(rgb_color[:3])  # Keep only RGB channels
        colors.append(group_colors)
    return colors

    
def draw_circles_with_text(frame, positions, colors, texts=None, text_color=(255, 255, 255),
                           radius=10, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.5, thickness=1, img_scale=1, score=None):
    # Convert frame to RGB format (OpenCV uses BGR by default)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if score is None:
        score = [None]*len(frame)
    for ind, ((x, y), color, prob) in enumerate(zip(positions, colors, score)):
        
  
        
        # Convert the color from 0-255 range to tuple format (BGR)
        b, g, r = color
        
        x = x*img_scale
        y = y*img_scale
        
        x = int(x)
        y = int(y)
        
        if prob is None:
            prob_txt = ''
        else:
            prob_txt = f'{prob:.2f}'
                
            
        circle_color = (b*255, g*255, r*255)
        
        
        # Draw circle on the frame
        cv2.circle(frame_rgb, (x, y), radius, circle_color, -1)
        
        if texts is not None:
            text_pos = (x - radius, y - radius - 10)
            # Draw text on the frame
            text = texts[ind]+prob_txt
            
            frame_bgr = cv2.putText(frame_rgb, text, text_pos, font,
                        font_scale, text_color, thickness, lineType=cv2.LINE_AA)


          
    # Convert frame back to BGR format for display
    frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
    return frame_bgr
    

    
def display_label_on_video(frame, index, labels_array,new_label=None,
                           display_label=True, display_map_flag=True, display_label_location=None, save_label_flag=False):
    height, width = frame.shape[:2]
       
        
    if not new_label is None and is_char_keycode(new_label):
        new_label = chr(new_label)            

        if new_label in action_map:
            labels_array[index] = new_label
            
    else:
        if index > 0 and save_label_flag:
            labels_array[index] = labels_array[index-1]
        
    
    if display_label_location is None:
        display_label_location = (width//2, height//2 + 100)
    
    label = labels_array[index]    
    if display_label:
        label = action_map[label]
        
#     cv2.putText(frame, f"Cluster: {label}", (width//2, height//2 + 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    h = 160
    if display_map_flag:
                
        h_increment = 30
        for key, val in action_display.items():
            if key == 'save mode':
                continue
            cv2.putText(frame, f"{key}: {val}", (20,h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            h += h_increment
    cv2.putText(frame, f"Save mode: {action_display['save mode']}", (20,h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    return labels_array
    
def is_char_keycode(key_code):
    # Check if the key code corresponds to a printable ASCII character
    return 32 <= key_code <= 126

def ensure_folder_for_file(file_path):
    folder_path = os.path.dirname(file_path)
    
    if not os.path.exists(folder_path):
        try:
            os.makedirs(folder_path)
            print(f"Folder '{folder_path}' created successfully.")
        except OSError as e:
            print(f"Error creating folder '{folder_path}': {e}")
    
    
def play_clustered_video(frames_in, df_pose, columns_to_plot,
                         labels_array=None, additional_variables=None, 
                         show_class=None, enable_labeling=False, save_flag=True, img_scale=1, df_score=None,
                         radius=5):
    print(radius)
    try:
        if isinstance(labels_array, str):
            save_file_path = labels_array
            labels_array = get_array_from_csv_file(labels_array)
        else:
            save_file_path = None              
            
        if labels_array is None:
            labels_array = ['-']*len(frames_in)
            
        action_display['save mode'] = False
        abs_fps = 29
        frame = frames_in[0].copy()
        frame = cv2.resize(frame, (0,0), fx=img_scale, fy=img_scale) 
        height, width = frame.shape[:2]

        pos_1_x = df_pose[[a for a in columns_to_plot if '1' in a and '_x' in a]].values
        pos_1_y = df_pose[[a for a in columns_to_plot if '1' in a and '_y' in a]].values
        
    
        

        pos_2_x = df_pose[[a for a in columns_to_plot if '2' in a and '_x' in a]].values
        pos_2_y = df_pose[[a for a in columns_to_plot if '2' in a and '_y' in a]].values
        
        if not df_score is None:
            prob_columns_to_plot = [p.replace('_x', '_p') for p in columns_to_plot if '_x' in p]
            pos_1_p = df_score[[a for a in prob_columns_to_plot if '1' in a]].values
            pos_2_p = df_score[[a for a in prob_columns_to_plot if '2' in a]].values
       

        colnames_1_x =   [a for a in columns_to_plot if '1' in a and '_x' in a]
        colnames_2_x =   [a for a in columns_to_plot if '2' in a and '_x' in a]


        colors = generate_distinct_colors(2, len(colnames_1_x))


        frame_number = 0
        n_frames = len(frames)
        pause_vid = False

        pause_fliker_counter = 0
        fps = abs_fps

        display_map_flag = True
        save_label_flag = action_display['save mode']

        prev_key = -1
        continuous_presss = 1
        continuous_presss_factor = 1
        
        while frame_number < len(frames_in):
            
            
            # Get user input

            key = cv2.waitKeyEx(int(1000 / fps))  # Display each frame for the appropriate time

            if key == prev_key and key !=-1 and not pause_vid:
                continuous_presss += 1
            else:
                continuous_presss = 1

#             if key !=-1:
#                 if is_char_keycode(key):
#                     print(chr(key))
#                 else:
#                     print(key)
            
            

            if continuous_presss==1:
                if key ==  ord('x'):
                    display_map_flag = not display_map_flag

                elif key == ord('c'):
                    action_display['save mode'] = not action_display['save mode']
                    save_label_flag = action_display['save mode']

            continuous_presss_factor = 1+continuous_presss/fps*0.5


            prev_key = key
            
            if pause_vid:
                shift_size = 1
            else:
                shift_size = int(1*continuous_presss)
            
            if key == 27:  # Press 'Esc' to exit
                break
            elif key == 2424832: # Press 'Left Arrow' to go back 10 frames
                frame_number = np.max([0, frame_number-shift_size])
            elif key == 2555904:               
                frame_number = np.min([n_frames, frame_number+shift_size]) # Press 'Right Arrow' to skip 10 frames
            elif key == 2490368 : # Predd "up" to speed up
                fps = np.min([100, fps+int(1*continuous_presss)])
            elif key == 2621440: # Predd "down" to slow down
                fps = np.max([1, fps-int(1*continuous_presss)])
            elif key == ord('z'): # Predd press ] to reset
                fps = abs_fps

            elif key == 32: # Pasue video
                pause_vid = not pause_vid
#                 pause_fliker_counter = 0
                st = time.time()
                frame_number = np.max([frame_number-1,0])
                

            
            frame_number = 0 if frame_number<0 else frame_number
            frame = frames_in[frame_number].copy()
            #Rescale frame and refresh frame dim
            frame = cv2.resize(frame, (0,0), fx=img_scale, fy=img_scale) 
            height, width = frame.shape[:2]
            
                        
            labels_array = display_label_on_video(frame, frame_number, labels_array,new_label=key,
                                   display_label=True, display_map_flag=display_map_flag, display_label_location=None,
                                  save_label_flag=save_label_flag and not pause_vid)       
            


            cluster = labels_array[frame_number]
            time_vid = frame_number / abs_fps

            if show_class is not None:
                while show_class != cluster and frame_number<n_frames:
                    frame_number += 1
                    cluster = cluster_labels[frame_number]

            if frame_number > n_frames:
                break      

            cv2.putText(frame, f"fps: {fps}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, f"Frame: {frame_number}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, f"Time: {time_vid:.2f} s", (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    #         cv2.putText(frame, f"Cluster: {cluster}", (width//2, height//2 + 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            if continuous_presss>1:
                cv2.putText(frame, f"Factor: {int(continuous_presss_factor)}",
                            (20, 700), cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 0, 255), 2)
            if additional_variables:
                h = 160
                for var in additional_variables:
                    value = np.round(df_pose.iloc[frame_number][var])

                    cv2.putText(frame, f"{var}: {value:0.0f}", (150, h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    h += 30

            # Draw Pose

            x = pos_1_x[frame_number,:]
            y = pos_1_y[frame_number,:]
            
#                 # debug
#             print(x, y)
            pos_1 = list(zip(x,y))
            if df_score is None:
                score_1 = None
            else:
                score_1 = pos_1_p[frame_number,:]
            
            frame = draw_circles_with_text(frame, pos_1, colors[0], texts=colnames_1_x, img_scale=img_scale, score=score_1,
                                           radius=radius)

            x = pos_2_x[frame_number,:]
            y = pos_2_y[frame_number,:]
            if df_score is None:
                score_2 = None
            else:
                score_2 = pos_2_p[frame_number,:]
            pos_2 = list(zip(x,y))
            frame = draw_circles_with_text(frame, pos_2, colors[1], texts=colnames_2_x, img_scale=img_scale, score=score_2, 
                                           radius=radius)


            # Display
            cv2.imshow("Clustered Video", frame)
            
            if pause_vid:
                frame_temp = frame.copy()
                cv2.putText(frame_temp, f"Video Paused", (width//2, height//2), cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 0, 255), 2)

                if int(time.time() - st) % 2 > 0:
                    cv2.imshow("Clustered Video", frame_temp)
                else:
                    cv2.imshow("Clustered Video", frame)
                pause_fliker_counter +=1
            else:
                frame_number+=1

            if cv2.getWindowProperty("Clustered Video", cv2.WND_PROP_VISIBLE) <1:
                break
        cv2.destroyAllWindows()
    except Exception as e:
        traceback_str = traceback.format_exc()
        print(f"An error occurred: {traceback_str}")
    finally:
        cv2.destroyAllWindows()
        
#         if save_file_path:
#             ensure_folder_for_file(save_file_path)
#             df_array = pd.DataFrame.from_dict({'frame_number': range(1,n_frames+1), 'action': labels_array})
#             df_array.to_csv(save_file_path)
#             print(f'Data was saved to file {save_file_path}')
#         return labels_array
    
    if save_file_path and save_flag:
        ensure_folder_for_file(save_file_path)
        df_array = pd.DataFrame.from_dict({
            'frame_number': range(1,len(frames_in)+1),
            'action': labels_array,
            'action_text': [action_map[str(a)] for a in labels_array]
        })
        df_array.to_csv(save_file_path)
        print(f'Data was saved to file {save_file_path}')
    return labels_array
    


In [13]:
# frames = load_video_frames(video_path)

In [14]:
# video_path = 'cropped_Rat1 probe 4-day2-free_2019-05-13-114301-0000.avi/'
# video_path = df_full_path.replace('csv','avi')
if 'frames' not in globals():
    frames = load_video_frames(video_path)

In [15]:
consol_actions = {'z':'Reset fps',
                       'x':'Toggle display',
                        'c':'Save labels',
                        'save mode': False,
                       'escape':'Close window'}
action_display = action_map.copy()
action_display.update(consol_actions)

                       

In [16]:
# save_path = df_full_path.replace('.csv', '_annotation.csv')
save_path = None

In [17]:
# save_path
save_flag = False
img_scale = 0.5
# labels = smooth_time_series(labels, x=30, y=500)
joints_to_plot = [a for a in all_joints if '_derivative' not in a]
# nose_columns = [a for a in joints_to_plot if 'Nose' in a]

In [18]:
df_original_pose = df_original[joints_to_plot]

In [19]:
# ind = 1
# joint = joint_names[0]
# window_length = 10
# polyorder=3
# data = df_pose.loc[:, joints_to_plot[0]]
# smoothed_data = savgol_filter(data, window_length=window_length, polyorder=polyorder)
# smoother = lambda x: savgol_filter(x, window_length=window_length, polyorder=polyorder)

In [20]:
# smoothed_pose_data = df_pose.apply(smoother)

In [24]:
labels_array = play_clustered_video(frames_in=frames, df_pose=df_original_pose, columns_to_plot=joints_to_plot,
                         labels_array=save_path, additional_variables=None, 
                         show_class=None, enable_labeling=True, save_flag=save_flag, img_scale=img_scale, df_score=df_score)


5


In [22]:


labels_array = play_clustered_video(frames_in=frames, df_pose=df_pose, columns_to_plot=joints_to_plot,
                         labels_array=save_path, additional_variables=None, 
                         show_class=None, enable_labeling=True, save_flag=save_flag, img_scale=img_scale, df_score=df_score)


5


In [ ]:
# map_dict = {
#         "Tail_end_1_x":"Tail_base_1_x",
#         "Lat_right_1_x":"Lateral_right_1_x",
#         "Tail_base_1_x":"Lateral_left_1_x",
#         "Lat_left_1_x":"Trunk",
#     }
# map_dict.items()  

In [ ]:
frames[0].shape

## FIX LABELS 

In [ ]:
cwd = os.getcwd()
os.chdir('..')
from utils.fix_dlc_predictions import predict_joint_pos, load_models
models, scalers = load_models('utils')
from utils.consts import NODES, NODES2IND
os.chdir(cwd)


In [ ]:
df_pose_reconstructed = df_pose.copy()

In [ ]:
NODES

In [ ]:
node_column_names_map = {
    'Ear_left' : 'Ear_left',
    'Ear_right' : 'Ear_right',
    'Nose' : 'Nose',
    'Neck' : 'Center',
    'Trunk' : 'Tail_base',
    'Lateral_right' : 'Lat_right',
    'Lateral_left' : 'Tail_end',
    'Tail_base' : 'Lat_left'
}



node_column_names_map_inv = {v:a for a,v in node_column_names_map.items()}
node_names = list(node_column_names_map.values())

In [ ]:
node_names

In [ ]:
def reconstsruct_df(df_in, models, scalers, df_score=None, gamma=0.5, frames=None):
    df_pose_reconstructed = df_in.copy()
    col_names = list(df_pose_reconstructed.columns)
    if frames is None:
        frames = range(len(df_pose))
    for frame in frames:

        for pair_ind in range(1, 3):
            joint_set = np.zeros((2, 8))
            probs = np.zeros((1,8))
            # get set
            for node in node_names:
                indices = [f'{node}_{pair_ind}_x', f'{node}_{pair_ind}_y']
                if df_score is None:
                    probs[:,NODES2IND[node_column_names_map_inv[node]]] = 0.5
                else:
                    prob_col_name = f'{node}_{pair_ind}_p'
                    probs[:,NODES2IND[node_column_names_map_inv[node]]] = df_score.loc[frame, prob_col_name]
                
                joint_set[:, NODES2IND[node_column_names_map_inv[node]]] = df_pose_reconstructed.loc[frame, indices].values
            # transform set
            _, _, reconstructed_joint_set = predict_joint_pos(joint_set, models, scalers)
            reconstructed_joint_set = reconstructed_joint_set.T  # to match input
            mean_values = (1-probs) * (1-gamma) *  reconstructed_joint_set + probs * gamma * joint_set
    #         mean_values = (1-gamma) * reconstructed_joint_set + gamma * joint_set

            # assign in table
            for node in node_names:
                indice_names = [f'{node}_{pair_ind}_x', f'{node}_{pair_ind}_y']
                indices = [col_names.index(a) for a in indice_names]

                df_pose_reconstructed.iloc[frame, indices] = mean_values[:, NODES2IND[node_column_names_map_inv[node]]] 
    return df_pose_reconstructed

In [ ]:
probs

In [ ]:
labels_array = play_clustered_video(frames_in=frames, df_pose=df_pose_reconstructed, columns_to_plot=joints_to_plot,
                         labels_array=save_path, additional_variables=None, 
                         show_class=None, enable_labeling=True, save_flag=save_flag, img_scale=1, df_score=df_score)


In [ ]:
labels_array = play_clustered_video(frames_in=frames, df_pose=df_pose, columns_to_plot=joints_to_plot,
                         labels_array=save_path, additional_variables=None, 
                         show_class=None, enable_labeling=True, save_flag=save_flag, img_scale=1, df_score=df_score)


In [ ]:
cols = ['Nose_1_x','Nose_1_y']
print(df_pose_reconstructed[cols].values[1678])
print(df_pose[cols].values[1678])
print(df_score[cols[0].replace('_x', '_p')].values[1678])

In [ ]:
cols = ['Nose_1_x','Nose_1_y']
print(df_pose_reconstructed[cols].values[1678])
print(df_pose[cols].values[1678])
print(df_score[cols[0].replace('_x', '_p')].values[1678])

In [ ]:
np.round(df_pose['Tail_base_1_y'].values) == np.round(df_pose_reconstructed['Tail_base_1_y'].values)